## Importing Python Libraries for EDA Analysis

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

Reading Data From CSV file

In [ ]:
df = pd.DataFrame(pd.read_csv('filename.csv'))

Retrieving Info of the DataFrame/DataSet

Shape of the DataSet

In [ ]:
df.shape()

Checking Missing Values From the DataSet

Checking for Duplicate Rows in DataSet

Dropping the Duplicates from teh DataSet and Filling the Null Values

Checking the Number of Unique Values in DataSet

Checking the Statistics of the DataSet

In [ ]:
df.describe()

## Exploring the DataSet ##
Checking for total Numerical and Categgorical Columns in the DataSet

Adding Any Columns if needed for analysis

## Data Visualization ##

Histogram & KDE

MultiVariate Analysis

UniVariate Analysis

BiVariate Analysis

Checking Outliers

MultiVariate Analysis Using Pairplot